In [14]:
# Imports

import numpy as np
import pandas as pd
from tqdm import tqdm

import pybel as bel
from sklearn.model_selection import train_test_split
from imblearn.over_sampling import RandomOverSampler
from sklearn.naive_bayes import GaussianNB


In [2]:
# Read molecules from sdf file

Molecules = []
for molecule in tqdm(bel.readfile("sdf", "naive_bayes.sdf")):
    Molecules.append(molecule)


290893it [01:55, 2526.74it/s]


In [3]:
# Read the activityscore from the CSV file. Compounds with inhibition above 15 are considered active. 

y = pd.read_csv("naive_bayes.sdf.csv")["PUBCHEM_ACTIVITY_SCORE"]

# Attempt 1
* Features: Circular fingerprints
* Classification method: 


In [5]:
X = []
for molecule in tqdm(Molecules):
    X.append(molecule.calcfp().bits)

100%|██████████| 290893/290893 [05:05<00:00, 950.64it/s] 


In [7]:
X_bitlist = [[False]*1025 for i in range(len(X))]

In [8]:
for i in range(len(X)):
    for j in X[i]:
        X_bitlist[i][j] = True

In [9]:
y_active = [1 if i >= 15 else 0 for i in y]

In [10]:
print(y_active.count(0), y_active.count(1))

290449 444


In [11]:
X_train, X_test, y_train, y_test = train_test_split(X_bitlist, y_active, test_size = 0.2, stratify = y_active)

In [12]:
# Resampling because of class imbalance
ros = RandomOverSampler(random_state = 0)
X_resampled, y_resampled = ros.fit_resample(X_train, y_train)

In [13]:
print(y_resampled.count(0), y_resampled.count(1))


232359 232359


In [ ]:
# Implementing the Naive Bayes
gnb = GaussianNB()
y_pred = gnb.fit(X_train, y_train).predict(X_test)